<a href="https://colab.research.google.com/github/Sans-codes/2462364_SanskritiAryal/blob/main/Worksheet6_SanskritiAryal_5CS037_ConceptsAndTechnologiesOfAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Implementation of Sigmoid Function:

In [2]:
def logistic_function(x):
    """
    Computes the logistic function applied to any value of x.
    Arguments:
    x: scalar or numpy array of any size.
    Returns:
    y: logistic function applied to x.
    """
    import numpy as np
    y = 1 / (1 + np.exp(-x))
    return y

Test Case for logistic function:

In [3]:
import numpy as np
def test_logistic_function():
    """
    Test cases for the logistic_function.
    """
    x_scalar = 0
    expected_output_scalar = round(1 / (1 + np.exp(0)), 3)
    assert round(logistic_function(x_scalar), 3) == expected_output_scalar

    x_pos = 2
    expected_output_pos = round(1 / (1 + np.exp(-2)), 3)
    assert round(logistic_function(x_pos), 3) == expected_output_pos

    x_neg = -3
    expected_output_neg = round(1 / (1 + np.exp(3)), 3)
    assert round(logistic_function(x_neg), 3) == expected_output_neg

    x_array = np.array([0, 2, -3])
    expected_output_array = np.array([0.5, 0.881, 0.047])
    assert np.all(np.round(logistic_function(x_array), 3) == expected_output_array)

    print("All tests passed!")

test_logistic_function()


All tests passed!


Implementation of log - loss function:

In [4]:
def log_loss(y_true, y_pred):
    """
    Computes log loss for true target value y ={0 or 1} and predicted target value y’ inbetween {0-1}.
    """
    import numpy as np
    y_pred = np.clip(y_pred, 1e-10, 1 - 1e-10)
    loss = -(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))
    return loss


Intuition verification

In [5]:
y_true, y_pred = 0, 0.1
print(f'log loss({y_true}, {y_pred}) ==> {log_loss(y_true, y_pred)}')
print("+++++++++++++--------------------------++++++++++++++++++++++++")
y_true, y_pred = 1, 0.9
print(f'log loss({y_true}, {y_pred}) ==> {log_loss(y_true, y_pred)}')


log loss(0, 0.1) ==> 0.10536051565782628
+++++++++++++--------------------------++++++++++++++++++++++++
log loss(1, 0.9) ==> 0.10536051565782628


Test case

In [6]:
def test_log_loss():
    import numpy as np
    y_true, y_pred = 1, 1
    assert np.isclose(log_loss(y_true, y_pred), 0.0)

    y_true, y_pred = 0, 0
    assert np.isclose(log_loss(y_true, y_pred), 0.0)

    y_true, y_pred = 1, 0.8
    expected_loss = -(np.log(0.8))
    assert np.isclose(log_loss(y_true, y_pred), expected_loss, atol=1e-6)

    y_true, y_pred = 0, 0.2
    expected_loss = -(np.log(0.8))
    assert np.isclose(log_loss(y_true, y_pred), expected_loss, atol=1e-6)

    print("All tests passed!")

test_log_loss()

All tests passed!


Cost Function

In [7]:
def cost_function(y_true, y_pred):
    """
    Computes log loss for inputs true value (0 or 1) and predicted value (between 0 and 1)
    """
    import numpy as np
    assert len(y_true) == len(y_pred)
    n = len(y_true)
    loss_vec = log_loss(y_true, y_pred)
    cost = np.sum(loss_vec) / n
    return cost

Test

In [8]:
import numpy as np
def test_cost_function():
    y_true = np.array([1, 0, 1])
    y_pred = np.array([0.9, 0.1, 0.8])

    expected_cost = (-(np.log(0.9)) - np.log(0.9) - np.log(0.8)) / 3
    result = cost_function(y_true, y_pred)

    assert np.isclose(result, expected_cost, atol=1e-6)
    print("Test passed!")

test_cost_function()

Test passed!


Vectorized Cost Function

In [9]:
def costfunction_logreg(X, y, w, b):
    import numpy as np
    n, d = X.shape
    z = np.dot(X, w) + b
    y_pred = logistic_function(z)
    cost = cost_function(y, y_pred)
    return cost

Compute Gradient

In [10]:
def compute_gradient(X, y, w, b):
    import numpy as np
    n, d = X.shape
    z = np.dot(X, w) + b
    y_pred = logistic_function(z)

    grad_w = np.dot(X.T, (y_pred - y)) / n
    grad_b = np.sum(y_pred - y) / n

    return grad_w, grad_b

Gradient Descent

In [11]:
def gradient_descent(X, y, w, b, alpha, n_iter, show_cost=False, show_params=True):
    n, d = X.shape
    cost_history = []
    params_history = []

    for i in range(n_iter):
        grad_w, grad_b = compute_gradient(X, y, w, b)
        w -= alpha * grad_w
        b -= alpha * grad_b

        cost = costfunction_logreg(X, y, w, b)
        cost_history.append(cost)
        params_history.append((w.copy(), b))

        if show_cost and (i % 100 == 0 or i == n_iter - 1):
            print(f"Iteration {i}: Cost = {cost:.6f}")

    return w, b, cost_history, params_history

Prediction Function

In [12]:
def prediction(X, w, b, threshold=0.5):
    import numpy as np
    z = np.dot(X, w) + b
    y_test_prob = logistic_function(z)
    y_pred = (y_test_prob >= threshold).astype(int)
    return y_pred


Evaluation Function

In [13]:
def evaluate_classification(y_true, y_pred):
    import numpy as np
    TP = np.sum((y_true == 1) & (y_pred == 1))
    TN = np.sum((y_true == 0) & (y_pred == 0))
    FP = np.sum((y_true == 0) & (y_pred == 1))
    FN = np.sum((y_true == 1) & (y_pred == 0))

    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0
    f1_score = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    return {
        "confusion_matrix": np.array([[TN, FP], [FN, TP]]),
        "precision": precision,
        "recall": recall,
        "f1_score": f1_score
    }

PART II SOFTMAX REGRESSION

Implement the Softmax Function

In [14]:
def softmax(z):
    """
    Compute the softmax of a 2D numpy array along the specified axis.
    """
    exp_z = np.exp(z - np.max(z, axis=1, keepdims=True))
    return exp_z / np.sum(exp_z, axis=1, keepdims=True)

In [15]:
z = np.array([[1, 2, 3]])
y = softmax(z)

print("Softmax output:", y)
print("Sum of probabilities:", np.sum(y))

Softmax output: [[0.09003057 0.24472847 0.66524096]]
Sum of probabilities: 0.9999999999999999


Softmax Loss Function

In [16]:
def loss_softmax(y_true, y_pred):
    return -np.sum(y_true * np.log(y_pred + 1e-10))

In [17]:
y_true = np.array([[0, 0, 1]])
y_pred = np.array([[0.1, 0.2, 0.7]])
print("Softmax Loss:", loss_softmax(y_true, y_pred))

Softmax Loss: 0.3566749437958753


Softmax Cost Function

In [18]:
def cost_softmax(X, y, W, b):
    n, d = X.shape
    z = np.dot(X, W) + b
    y_pred = softmax(z)
    return -np.sum(y * np.log(y_pred + 1e-10)) / n

In [19]:
X = np.array([[1, 2]])
y = np.array([[0, 1]])
W = np.array([[0.1, 0.2],
              [0.3, 0.4]])
b = np.array([0.1, 0.1])

print("Softmax Cost:", cost_softmax(X, y, W, b))

Softmax Cost: 0.5543552442944452


PART II VERIFICATION:
The softmax function was verified using a sample input.
The output probabilities are between 0 and 1, and their sum equals 1.
The loss and cost functions produce valid numerical outputs, confirming correct implementation.
